In [8]:
import argparse
import logging
import os
import time

import dgl
import dgl.function as fn
import dgl.nn.pytorch as dglnn
import networkx as nx
import numpy as np
import pandas as pd
import scipy.sparse as ssp
import torch
import torch.nn as nn
from tqdm import trange
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

In [9]:
import sys
sys.path.append("/nfs/zty/Graph/Dynamic-Graph")
sys.path.append(".")
from data_loader.minibatch import load_data
from model.utils import get_free_gpu

In [10]:
import importlib
import util_dgl
importlib.reload(util_dgl)
# from util_dgl import test_graph, construct_dglgraph

INFO:root:Namespace(gpu=True)


<module 'util_dgl' from '/nfs/zty/Graph/Dynamic-Graph/torch_model/util_dgl.py'>

In [11]:
device = torch.device("cpu")
edges, nodes = util_dgl.test_graph()
g = util_dgl.construct_dglgraph(edges, nodes, device)

# logger.info(f"Begin Conv. Device {device}")
dim = g.ndata["nfeat"].shape[-1]
dims = [dim, 108, 4]
# for l in range(1, 3):
l = 1
# logger.info(f"Graph Conv Layer {l}.")
model = util_dgl.TSAGEConv(in_feats=dims[l-1], out_feats=dims[l], aggregator_type="mean")
model = model.to(device)
src_feat, dst_feat = model(g, current_layer=l)
g.edata[f"src_feat{l}"] = src_feat
g.edata[f"dst_feat{l}"] = dst_feat

In [12]:
print(model)

TSAGEConv(
  (feat_drop): Dropout(p=0.0, inplace=False)
  (fc_self): Linear(in_features=4, out_features=108, bias=True)
  (fc_neigh): Linear(in_features=4, out_features=108, bias=True)
)


In [13]:
from torchviz import make_dot
dot = make_dot(src_feat)

In [14]:
dot.render('two-layer-conv', view=True) 

DEBUG:graphviz.files:write 18625 bytes to 'two-layer-conv'
DEBUG:graphviz.backend:run ['dot', '-Tpdf', '-O', 'two-layer-conv']
DEBUG:graphviz.backend:view: ['xdg-open', 'two-layer-conv.pdf']


'two-layer-conv.pdf'